### Setups

In [1]:
import numpy as np
from pyscf import ao2mo, gto, scf, lib
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit_aer.primitives import Estimator as AerEstimator
from scipy.optimize import minimize
from qiskit_nature.second_q.circuit.library import UCC
# from qiskit_nature.drivers import PySCFDriver
# from qiskit_nature.converters.second_quantization import QubitConverter
# from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
# from qiskit_nature.mappers.second_quantization import ParityMapper
# from qiskit_nature.circuit.library.ansatzes import UCCSD
from qiskit.quantum_info.analysis import Z2Symmetries
import matplotlib.pyplot as plt
from pyscf.gto import Mole
from pyscf.lib import logger
from pyscf.geomopt import berny_solver
from asf.wrapper import find_from_mol, sized_space_from_mol, find_from_scf
from asf.scf import stable_scf

In [2]:
# Property for figures
plt.rcParams['font.family'] = "Times New Roman"
plt.rcParams['xtick.direction'] = "in"
plt.rcParams['ytick.direction'] = "in"
plt.rcParams['xtick.major.width'] = 1.0
plt.rcParams['ytick.major.width'] = 1.0
plt.rcParams['xtick.minor.width'] = 0.5
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['xtick.minor.size'] = 2
plt.rcParams['font.size'] = 20
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.linewidth'] = 1.0

## Define Function

In [3]:
def create_integrals_ib_sto3g():
    # Li3Co3O6(gas phase)
    mol = gto.M(
        atom = '''
        Li  0.0000  0.0000  0.0000
        Co  0.0000  0.0000  2.10
        O   1.25   0.0000  3.20
        O  -1.25   0.0000  3.20
        ''', 
        basis = '6-31G*'
    )
    # do the HF calculation
    hf = scf.RHF(mol)
    hf.kernel()
    # number of electrons (N)
    nelec = mol.nelectron
    # number of orbitals (M)
    norb = hf.mo_coeff.shape[-1]
    # orbital coefficients (C_ji)
    c = hf.mo_coeff
    print(f"nOrb : {norb}")
    print(f"nElec: {nelec}")
    # nuclear repulsion energy E_II
    nuclear = mol.energy_nuc()
    # h^(atom)_ab
    h1atom = hf.get_hcore()
    # hpq = sum_ab C_ap^* C_bq h^(atom)_ab
    # since c is all real (no complex number), we don’t need to conjugate
    hpq = np.einsum("ap,bq,ab->pq", c, c, h1atom)
    use_slow_version = False
    if use_slow_version:
      # SLOW VERSION OF hpqrs
      # hpqrs = sum_abcd C_ap^* C_bq^* C_cr C_ds h^(atom)_abcd
      # PYSCF USES CHEMISTS NOTATION, SO WE NEED TO DO
      # (ps|qr) = sum_adbc C_ap^* C_bq^* C_cr C_ds (ad|bc)^((atom))
      # since c is all real (no complex number), we don’t need to conjugate
      h2atom = mol.intor("int2e")
      hpqrs = np.einsum("ap,bq,cr,ds,adbc->pqrs", c, c, c, c, h2atom)
    else:
      # FASTER VERSION OF hpqrs
      # PYSCF USES CHEMISTS NOTATION, NEED TO CHANGE THE ORDER
      # (ps|qr) -> h_pqrs
      hpqrs = ao2mo.full(mol, c, compact=False).reshape(norb, norb, norb, norb)
      hpqrs = hpqrs.transpose(0, 2, 3, 1)
    return hpq, hpqrs, nuclear, nelec, hf

In [4]:
def create_integrals_CoO2_sto3g():
    # CoO2
    mol = gto.M(
        atom = '''
        Co  0.0000  0.0000  2.10
        O   1.25   0.0000  3.20
        O  -1.25   0.0000  3.20
        ''', 
        basis = '6-31G*'
    )
    # do the HF calculation
    hf = scf.RHF(mol)
    hf.kernel()
    # number of electrons (N)
    nelec = mol.nelectron
    # number of orbitals (M)
    norb = hf.mo_coeff.shape[-1]
    # orbital coefficients (C_ji)
    c = hf.mo_coeff
    print(f"nOrb : {norb}")
    print(f"nElec: {nelec}")
    # nuclear repulsion energy E_II
    nuclear = mol.energy_nuc()
    # h^(atom)_ab
    h1atom = hf.get_hcore()
    # hpq = sum_ab C_ap^* C_bq h^(atom)_ab
    # since c is all real (no complex number), we don’t need to conjugate
    hpq = np.einsum("ap,bq,ab->pq", c, c, h1atom)
    use_slow_version = False
    if use_slow_version:
      # SLOW VERSION OF hpqrs
      # hpqrs = sum_abcd C_ap^* C_bq^* C_cr C_ds h^(atom)_abcd
      # PYSCF USES CHEMISTS NOTATION, SO WE NEED TO DO
      # (ps|qr) = sum_adbc C_ap^* C_bq^* C_cr C_ds (ad|bc)^((atom))
      # since c is all real (no complex number), we don’t need to conjugate
      h2atom = mol.intor("int2e")
      hpqrs = np.einsum("ap,bq,cr,ds,adbc->pqrs", c, c, c, c, h2atom)
    else:
      # FASTER VERSION OF hpqrs
      # PYSCF USES CHEMISTS NOTATION, NEED TO CHANGE THE ORDER
      # (ps|qr) -> h_pqrs
      hpqrs = ao2mo.full(mol, c, compact=False).reshape(norb, norb, norb, norb)
      hpqrs = hpqrs.transpose(0, 2, 3, 1)
    return hpq, hpqrs, nuclear, nelec, hf

In [5]:
def create_integrals_c2h4_sto3g():
    # ethene(gas phase)
    ethene_geometry = """
        C        0.6695      0.0000      0.0000
        C       -0.6695      0.0000      0.0000
        H        1.2320      0.0000     -0.9289
        H        1.2320      0.0000      0.9289
        H       -1.2320      0.0000      0.9289
        H       -1.2320      0.0000     -0.9289
    """
    mol = gto.M(atom = ethene_geometry, basis = 'sto-3g')
    # do the HF calculation
    hf = scf.RHF(mol)
    hf.kernel()
    # number of electrons (N)
    nelec = mol.nelectron
    # number of orbitals (M)
    norb = hf.mo_coeff.shape[-1]
    # orbital coefficients (C_ji)
    c = hf.mo_coeff
    print(f"nOrb : {norb}")
    print(f"nElec: {nelec}")
    # nuclear repulsion energy E_II
    nuclear = mol.energy_nuc()
    # h^(atom)_ab
    h1atom = hf.get_hcore()
    # hpq = sum_ab C_ap^* C_bq h^(atom)_ab
    # since c is all real (no complex number), we don’t need to conjugate
    hpq = np.einsum("ap,bq,ab->pq", c, c, h1atom)
    # PYSCF USES CHEMISTS NOTATION, NEED TO CHANGE THE ORDER
    # (ps|qr) -> h_pqrs
    hpqrs = ao2mo.full(mol, c, compact=False).reshape(norb, norb, norb, norb)
    hpqrs = hpqrs.transpose(0, 2, 3, 1)
    return hpq, hpqrs, nuclear, nelec, hf

In [6]:
def create_integrals_h2o_sto3g():
    water_geometry = """
        O  0.000000  0.000000  0.000000
        H  0.758602  0.000000  0.504284
        H  0.758602  0.000000  -0.504284
    """
    # water_geometry = """
    #     O  0.000000  0.000000  0.000000
    #     H  0.000000  -0.757000  0.586000
    #     H  0.000000  0.757000  0.586000
    # """
    mol = gto.M(atom = water_geometry, basis = 'sto-3g')
    # do the HF calculation
    hf = scf.RHF(mol)
    hf.kernel()
    # number of electrons (N)
    nelec = mol.nelectron
    # number of orbitals (M)
    norb = hf.mo_coeff.shape[-1]
    # orbital coefficients (C_ji)
    c = hf.mo_coeff
    print(f"nOrb : {norb}")
    print(f"nElec: {nelec}")
    # nuclear repulsion energy E_II
    nuclear = mol.energy_nuc()
    # h^(atom)_ab
    h1atom = hf.get_hcore()
    # hpq = sum_ab C_ap^* C_bq h^(atom)_ab
    # since c is all real (no complex number), we don’t need to conjugate
    hpq = np.einsum("ap,bq,ab->pq", c, c, h1atom)
    # PYSCF USES CHEMISTS NOTATION, NEED TO CHANGE THE ORDER
    # (ps|qr) -> h_pqrs
    hpqrs = ao2mo.full(mol, c, compact=False).reshape(norb, norb, norb, norb)
    hpqrs = hpqrs.transpose(0, 2, 3, 1)
    return hpq, hpqrs, nuclear, nelec, hf

In [7]:
def create_integrals_n2o4_sto3g():
    n2o4_geometry = """
        O  2.0000    0.2500    0.0000
        O  4.5981   -0.2500    0.0000
        O  2.8660   -1.2500    0.0000
        O  3.7320    1.2500    0.0000
        N  2.8660   -0.2500    0.0000
        N  3.7320    0.2500    0.0000
    """
    mol = gto.M(atom = n2o4_geometry, basis = 'sto-3g')
    # do the HF calculation
    hf = scf.RHF(mol)
    hf.kernel()
    # number of electrons (N)
    nelec = mol.nelectron
    # number of orbitals (M)
    norb = hf.mo_coeff.shape[-1]
    # orbital coefficients (C_ji)
    c = hf.mo_coeff
    print(f"nOrb : {norb}")
    print(f"nElec: {nelec}")
    # nuclear repulsion energy E_II
    nuclear = mol.energy_nuc()
    # h^(atom)_ab
    h1atom = hf.get_hcore()
    # hpq = sum_ab C_ap^* C_bq h^(atom)_ab
    # since c is all real (no complex number), we don’t need to conjugate
    hpq = np.einsum("ap,bq,ab->pq", c, c, h1atom)
    # PYSCF USES CHEMISTS NOTATION, NEED TO CHANGE THE ORDER
    # (ps|qr) -> h_pqrs
    hpqrs = ao2mo.full(mol, c, compact=False).reshape(norb, norb, norb, norb)
    hpqrs = hpqrs.transpose(0, 2, 3, 1)
    return hpq, hpqrs, nuclear, nelec, hf

In [8]:
def active_space_unpolarized(mf, ncore=0, nact=None):
    einsum = lib.einsum

    if nact is None:
        nact = mf.mo_coeff.shape[1] - ncore

    nuclear = mf.mol.energy_nuc()
    ecore = 0.0

    mo_coeff_core = mf.mo_coeff[:, :ncore]
    mo_coeff_acti = mf.mo_coeff[:, ncore : ncore + nact]

    h1atom = mf.get_hcore()
    if ncore != 0:
        dm_ao_core = 2 * mo_coeff_core @ mo_coeff_core.T
        vj, vk = mf.get_jk(mf.mol, dm_ao_core)
        veff_ao = vj - 0.5 * vk
        ecore += einsum("ij,ji->", dm_ao_core, h1atom + 0.5 * veff_ao)
        h1atom += veff_ao
    hpq = einsum("ap,bq,ab->pq", mo_coeff_acti, mo_coeff_acti, h1atom)
    hpqrs = ao2mo.full(mf.mol, mo_coeff_acti, compact=False).reshape(
        nact, nact, nact, nact
    )
    hpqrs = hpqrs.transpose(0, 2, 3, 1)

    act_nelec = mf.mol.nelectron - ncore * 2

    return nact, act_nelec, nuclear + ecore, hpq, hpqrs, nuclear

In [9]:
def create_hamiltonian_quantum_circuit(hpq, hpqrs, mapper):
    # NOTE: we exclude the nuclear term here.
    # We just add the nuclear term to the VQE energy afterwards
    n_spatial_orb = hpq.shape[0]
    n_spin_orb = 2 * n_spatial_orb
    fermionic_op_dict = {}

    for spin in range(2):
        # 0 is UP 1 is DW
        for p_spatial_orb in range(n_spatial_orb):
            for q_spatial_orb in range(n_spatial_orb):
                p_spin_orb = spin * n_spatial_orb + p_spatial_orb
                q_spin_orb = spin * n_spatial_orb + q_spatial_orb
                p_qubit = p_spin_orb
                q_qubit = q_spin_orb
                coeff = hpq[p_spatial_orb, q_spatial_orb]
                fermionic_op_dict[f"+_{p_qubit} -_{q_qubit}"] = coeff

    for spin_ps in range(2):
        for spin_qr in range(2):
            for p_spatial_orb in range(n_spatial_orb):
                for q_spatial_orb in range(n_spatial_orb):
                    for r_spatial_orb in range(n_spatial_orb):
                        for s_spatial_orb in range(n_spatial_orb):
                            p_spin_orb = spin_ps * n_spatial_orb + p_spatial_orb
                            q_spin_orb = spin_qr * n_spatial_orb + q_spatial_orb
                            r_spin_orb = spin_qr * n_spatial_orb + r_spatial_orb
                            s_spin_orb = spin_ps * n_spatial_orb + s_spatial_orb
                            p_qubit = p_spin_orb
                            q_qubit = q_spin_orb
                            r_qubit = r_spin_orb
                            s_qubit = s_spin_orb
                            coeff = hpqrs[p_spatial_orb, q_spatial_orb, r_spatial_orb, s_spatial_orb]
                            # multiply by 0.5
                            fermionic_op_dict[f"+_{p_qubit} +_{q_qubit} -_{r_qubit} -_{s_qubit}"] = (coeff * 0.5)

    fermionic_op = FermionicOp(fermionic_op_dict, num_spin_orbitals=n_spin_orb)
    qubit_op = mapper.map(fermionic_op)
    return qubit_op

In [10]:
def make_ucc_ansatz(n_spatial_orb, n_elec, mapper, print_circuit=False):
    n_spin_orb = n_spatial_orb * 2
    n_elec_per_spin = n_elec // 2
    # first create the HF state
    n_qubits = n_spin_orb
    hf = QuantumCircuit(n_qubits)

    for spin in range(2):
        # loop over occupied spatial orbitals
        for p_spatial_orb in range(n_elec_per_spin):
            p_spin_orb = spin * n_spatial_orb + p_spatial_orb
            hf.x(p_spin_orb)

    # num_particles: we need to separate number of spin up and spin down electrons
    # excitations: sd -> singles + doubles
    # qubit_mapper: The Jordan Wigner mapper
    # initial state is HF
    ucc = UCC(
        num_spatial_orbitals=n_spatial_orb,
        num_particles=(n_elec_per_spin, n_elec_per_spin),
        excitations="sd",
        qubit_mapper=mapper,
        initial_state=hf,
    )

    if print_circuit:
        ucc_draw = ucc.decompose().decompose().decompose()
        print(ucc_draw.draw("text"))

    return ucc

# UCCtest = make_ucc_ansatz(n_spatial_orb=arr_active[0], n_elec=2, mapper=JordanWignerMapper())
# UCCtest.decompose().decompose().decompose().draw("mpl")

In [11]:
def make_hea_ansatz(n_spatial_orb, reps=2, print_circuit=False):
    n_spin_orb = n_spatial_orb * 2
    n_qubits = n_spin_orb
    n_params = n_qubits * reps
    params = ParameterVector("t", n_params)
    ansatz = QuantumCircuit(n_qubits)
    iparam = 0

    for irep in range(reps):
        for p_spin_orb in range(n_spin_orb):
            ansatz.ry(params[iparam], p_spin_orb)
            iparam += 1
        ansatz.barrier()

        for p_spin_orb in range(n_spin_orb - 1):
            ansatz.cx(p_spin_orb, p_spin_orb + 1)
        ansatz.barrier()

    if print_circuit:
        ansatz_draw = ansatz.decompose()
        print(ansatz_draw.draw("text"))

    return ansatz
# HEAtest = make_hea_ansatz(n_spatial_orb=arr_active[0], reps=2, print_circuit=False)

In [12]:
def make_odd_hea_ansatz(n_spin, reps=2, print_circuit=False):
    n_spin_orb = n_spin
    n_qubits = n_spin_orb
    n_params = n_qubits * reps
    params = ParameterVector("t", n_params)
    ansatz = QuantumCircuit(n_qubits)
    iparam = 0

    for irep in range(reps):
        for p_spin_orb in range(n_spin_orb):
            ansatz.ry(params[iparam], p_spin_orb)
            iparam += 1
        ansatz.barrier()

        for p_spin_orb in range(n_spin_orb - 1):
            ansatz.cx(p_spin_orb, p_spin_orb + 1)
        ansatz.barrier()

    if print_circuit:
        ansatz_draw = ansatz.decompose()
        print(ansatz_draw.draw("text"))

    return ansatz
# HEAtest = make_hea_ansatz(n_spatial_orb=arr_active[0], reps=2, print_circuit=False)

In [13]:
def do_vqe(ansatz, hamiltonian, nuclear):
  nparams = ansatz.num_parameters
  # theta: set the inital theta to random
  initial_state = np.random.random(nparams)
  # noiseless simulator
  estimator = AerEstimator(run_options={"shots": None}, approximation=True)
  index = 0
  def get_energy(parameter_values):
    nonlocal index
    result = estimator.run(
      circuits=ansatz, observables=hamiltonian, parameter_values=parameter_values
    ).result()
    #print(ansatz.draw("mpl"))
    energy = result.values[0] + nuclear
    index += 1
    parameter_str = " ".join(f"{e:.4f}" for e in parameter_values)
    print(f"#{index:5d}: {energy:.6f}: {parameter_str}")
    return energy

  res = minimize(fun=get_energy, x0=initial_state, method="bfgs")
  print(f"Energy: {res.fun}")

# Optimize the molecular geometric 

In [14]:
mol = gto.M(
    atom=[
        ("Li", [0, 0, 0]),
        ("Co", [0, 0, 4.7]),
        ("O", [1.4, 0, 2.35]),
        ("O", [-1.4, 0, 2.35]),
    ],
    unit="Angstrom",
    basis="sto-3g",
)

In [15]:
mf = scf.RHF(mol)
mf.kernel()
mol_eq = berny_solver.optimize(mf, maxsteps=100)

print("Optimized atomic coordinates:")
print(mol_eq.atom_coords())

SCF not converged.
SCF energy = -1520.81054210662

Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
  Li   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
  Co   0.000000   0.000000   4.700000    0.000000  0.000000  0.000000
   O   1.400000   0.000000   2.350000    0.000000  0.000000  0.000000
   O  -1.400000   0.000000   2.350000    0.000000  0.000000  0.000000
converged SCF energy = -1520.81061736812
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 Li     0.0000000008     0.0000000000    -0.0659615988
1 Co    -0.0000000000     0.0000000000     0.0848065796
2 O     0.0312226266    -0.0000000000    -0.0094224877
3 O    -0.0312226274    -0.0000000000    -0.0094224931
----------------------------------------------
cycle 1: E = -1520.81061737  dE = -1520.81  norm(grad) = 0.11692

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Ato

# Calculate the active space of molecular

In [17]:
ethene_geometry = """
    C        0.6695      0.0000      0.0000
    C       -0.6695      0.0000      0.0000
    H        1.2320      0.0000     -0.9289
    H        1.2320      0.0000      0.9289
    H       -1.2320      0.0000      0.9289
    H       -1.2320      0.0000     -0.9289
"""
ethene_basis = "def2-SVP"
water_geometry = """
    O  0.000000  0.000000  0.000000
    H  0.758602  0.000000  0.504284
    H  0.758602  0.000000  -0.504284
"""
water_basis = "sto-g3"
n2o4_geometry = """
    O  2.0000    0.2500    0.0000
    O  4.5981   -0.2500    0.0000
    O  2.8660   -1.2500    0.0000
    O  3.7320    1.2500    0.0000
    N  2.8660   -0.2500    0.0000
    N  3.7320    0.2500    0.0000
"""
no2_geometry = """
    O  2.0000    0.2500    0.0000
    O  3.7320    0.2500    0.0000
    N  2.8660   -0.2500    0.0000
"""
mol = Mole(atom=water_geometry, unit="Angstrom" ,basis="def2-SVP", charge=0, spin=0, verbose=logger.INFO).build()
mf = scf.UHF(mol)
active_space = find_from_mol(mol)
print (active_space)

System: uname_result(system='Darwin', node='youdeMacBook-Air.local', release='23.6.0', version='Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:21 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T8103', machine='x86_64')  Threads 8
Python 3.11.9 (main, Apr 19 2024, 11:44:45) [Clang 14.0.6 ]
numpy 1.26.4  scipy 1.14.0  h5py 3.11.0
Date: Sat Aug 24 21:12:07 2024
PySCF version 2.6.2
PySCF path  /opt/anaconda3/envs/test_env/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = Angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.758602000000   0.000000000000  

In [ ]:
mol = Mole()
mol.build(
    atom = '''
    Li  0.0000  0.0000  0.0000
    Co  0.0000  0.0000  2.10
    O   1.25   0.0000  3.20
    O  -1.25   0.0000  3.20
    ''',
    basis="6-31g*",
)
mf = stable_scf(mol, with_uhf=False)
active_space = find_from_scf(mf)
print (active_space)

## Changing the Number of Active Cite

In [ ]:
arr = create_integrals_ib_sto3g()
# for i_act in range(5,7):
i_act = 8 # 12
arr_active = active_space_unpolarized(arr[4], ncore=16, nact=i_act)
hamiltonian = create_hamiltonian_quantum_circuit(arr_active[3], arr_active[4], JordanWignerMapper())
# HEAtest = make_hea_ansatz(n_spatial_orb=arr_active[0], reps=2, print_circuit=False)
# VQEtest = do_vqe(HEAtest, hamiltonian, arr_active[2])
print(len(hamiltonian))
print(hamiltonian)

converged SCF energy = -1537.93504404192
nOrb : 76
nElec: 46
2157
SparsePauliOp(['IIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIYZZY', 'IIIIIIIIIIIIXZZX', 'IIIIIIIIIIYZZZZY', 'IIIIIIIIIIXZZZZX', 'IIIIIIIIIYZZZZZY', 'IIIIIIIIIXZZZZZX', 'IIIIIIIIYZZZZZZY', 'IIIIIIIIXZZZZZZX', 'IIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIYZYI', 'IIIIIIIIIIIIXZXI', 'IIIIIIIIIIYZZZYI', 'IIIIIIIIIIXZZZXI', 'IIIIIIIIIYZZZZYI', 'IIIIIIIIIXZZZZXI', 'IIIIIIIIYZZZZZYI', 'IIIIIIIIXZZZZZXI', 'IIIIIIIIIIIIIZII', 'IIIIIIIIIIIYZYII', 'IIIIIIIIIIIXZXII', 'IIIIIIIIIIIIZIII', 'IIIIIIIIIIYZYIII', 'IIIIIIIIIIXZXIII', 'IIIIIIIIIYZZYIII', 'IIIIIIIIIXZZXIII', 'IIIIIIIIYZZZYIII', 'IIIIIIIIXZZZXIII', 'IIIIIIIIIIIZIIII', 'IIIIIIIIIIZIIIII', 'IIIIIIIIIYYIIIII', 'IIIIIIIIIXXIIIII', 'IIIIIIIIYZYIIIII', 'IIIIIIIIXZXIIIII', 'IIIIIIIIIZIIIIII', 'IIIIIIIIYYIIIIII', 'IIIIIIIIXXIIIIII', 'IIIIIIIIZIIIIIII', 'IIIIIIIZIIIIIIII', 'IIIIYZZYIIIIIIII', 'IIIIXZZXIIIIIIII', 'IIYZZZZYIIIIIIII', 'IIXZZZZXIIIIIIII', 'IYZZZZZYIIIIIIII', 'IXZZZZZXIIIIIIII',

In [ ]:
z2_symmetries = Z2Symmetries.find_z2_symmetries(hamiltonian)
tapered_op = z2_symmetries.taper(hamiltonian)
print(len(tapered_op))
print(len(tapered_op[1]))
print(tapered_op[1])

8
2648
SparsePauliOp(['IIIIIIIIIIIII', 'IIIZIZIZZZIZZ', 'IIIZIZIZZZIZX', 'IIIIIIIIIIIIX', 'IIIIIIIZZZZXI', 'IIIZIZIIIIZXZ', 'IIIIIIIZZXIII', 'IIIZIZIIIXIZZ', 'IIIIIIIZXIIII', 'IIIZIZIIXZIZZ', 'IIIIIIIXIIIII', 'IIIZIZIXZZIZZ', 'IIIIIIIIIIIIZ', 'IIIZIZIIIIZYY', 'IIIZIZIIIIZXX', 'IIIZIZIIIYIZY', 'IIIZIZIIIXIZX', 'IIIZIZIIYZIZY', 'IIIZIZIIXZIZX', 'IIIZIZIYZZIZY', 'IIIZIZIXZZIZX', 'IIIZIZIIIIZII', 'IIIZIZIIIIXZI', 'IIIIIIIIIIXZI', 'IIIIIIIIIIIZI', 'IIIIIIIIIYZYI', 'IIIIIIIIIXZXI', 'IIIIIIIIYZZYI', 'IIIIIIIIXZZXI', 'IIIIIIIYZZZYI', 'IIIIIIIXZZZXI', 'IIIIIIIIIIZII', 'IIIIIIIIIZIII', 'IIIIIIIIYYIII', 'IIIIIIIIXXIII', 'IIIIIIIYZYIII', 'IIIIIIIXZXIII', 'IIIIIIIIZIIII', 'IIIIIIIYYIIII', 'IIIIIIIXXIIII', 'IIIIIIIZIIIII', 'ZZZZZZZIIIIII', 'ZZZZZZXIIIIII', 'IIIIIIXIIIIII', 'ZZZZXIIIIIIII', 'IIIIXZZIIIIII', 'ZZXIIIIIIIIII', 'IIXZZZZIIIIII', 'ZXIIIIIIIIIII', 'IXZZZZZIIIIII', 'XIIIIIIIIIIII', 'XZZZZZZIIIIII', 'IIIIIIZIIIIII', 'IIIIYZYIIIIII', 'IIIIXZXIIIIII', 'IIYZZZYIIIIII', 'IIXZZZXIIIIII', 'IYZZZZYI

In [ ]:
HEAtest = make_odd_hea_ansatz(n_spin=i_act*2-3, reps=2, print_circuit=False)
VQEtest = do_vqe(HEAtest, tapered_op[6], arr_active[2])

#    1: -1523.195248: 0.0885 0.3637 0.6737 0.7136 0.1522 0.5578 0.9607 0.0459 0.2282 0.8418 0.7553 0.6886 0.0002 0.6677 0.4593 0.4935 0.6544 0.1361 0.3187 0.9451 0.5116 0.2982 0.1563 0.7654 0.9484 0.7157
#    2: -1523.195248: 0.0885 0.3637 0.6737 0.7136 0.1522 0.5578 0.9607 0.0459 0.2282 0.8418 0.7553 0.6886 0.0002 0.6677 0.4593 0.4935 0.6544 0.1361 0.3187 0.9451 0.5116 0.2982 0.1563 0.7654 0.9484 0.7157
#    3: -1523.195248: 0.0885 0.3637 0.6737 0.7136 0.1522 0.5578 0.9607 0.0459 0.2282 0.8418 0.7553 0.6886 0.0002 0.6677 0.4593 0.4935 0.6544 0.1361 0.3187 0.9451 0.5116 0.2982 0.1563 0.7654 0.9484 0.7157
#    4: -1523.195248: 0.0885 0.3637 0.6737 0.7136 0.1522 0.5578 0.9607 0.0459 0.2282 0.8418 0.7553 0.6886 0.0002 0.6677 0.4593 0.4935 0.6544 0.1361 0.3187 0.9451 0.5116 0.2982 0.1563 0.7654 0.9484 0.7157
#    5: -1523.195248: 0.0885 0.3637 0.6737 0.7136 0.1522 0.5578 0.9607 0.0459 0.2282 0.8418 0.7553 0.6886 0.0002 0.6677 0.4593 0.4935 0.6544 0.1361 0.3187 0.9451 0.5116 0.2982 0.1563 0

# Small molecular experiment

### H2O hea before tapering 

In [ ]:
hpq, hpqrs, nuclear, nelec, hf = create_integrals_h2o_sto3g()
hamiltonian = create_hamiltonian_quantum_circuit(hpq, hpqrs, JordanWignerMapper())
# HEAtest = make_hea_ansatz(n_spatial_orb=arr_active[0], reps=2, print_circuit=False)
# VQEtest = do_vqe(HEAtest, hamiltonian, arr_active[2])
print(len(hamiltonian))
print(hamiltonian)

converged SCF energy = -74.8801743517023
nOrb : 7
nElec: 10
1086
SparsePauliOp(['IIIIIIIIIIIIII', 'IIIIIIIIIIIIIZ', 'IIIIIIIIIIIIYY', 'IIIIIIIIIIIIXX', 'IIIIIIIIIIYZZY', 'IIIIIIIIIIXZZX', 'IIIIIIIIYZZZZY', 'IIIIIIIIXZZZZX', 'IIIIIIIIIIIIZI', 'IIIIIIIIIIYZYI', 'IIIIIIIIIIXZXI', 'IIIIIIIIYZZZYI', 'IIIIIIIIXZZZXI', 'IIIIIIIIIIIZII', 'IIIIIIIYZZZYII', 'IIIIIIIXZZZXII', 'IIIIIIIIIIZIII', 'IIIIIIIIYZYIII', 'IIIIIIIIXZXIII', 'IIIIIIIIIZIIII', 'IIIIIIIIZIIIII', 'IIIIIIIZIIIIII', 'IIIIIIZIIIIIII', 'IIIIIYYIIIIIII', 'IIIIIXXIIIIIII', 'IIIYZZYIIIIIII', 'IIIXZZXIIIIIII', 'IYZZZZYIIIIIII', 'IXZZZZXIIIIIII', 'IIIIIZIIIIIIII', 'IIIYZYIIIIIIII', 'IIIXZXIIIIIIII', 'IYZZZYIIIIIIII', 'IXZZZXIIIIIIII', 'IIIIZIIIIIIIII', 'YZZZYIIIIIIIII', 'XZZZXIIIIIIIII', 'IIIZIIIIIIIIII', 'IYZYIIIIIIIIII', 'IXZXIIIIIIIIII', 'IIZIIIIIIIIIII', 'IZIIIIIIIIIIII', 'ZIIIIIIIIIIIII', 'IIIIIIIIIIIIZZ', 'IIIIIIIIIIYZYZ', 'IIIIIIIIIIXZXZ', 'IIIIIIIIYZZZYZ', 'IIIIIIIIXZZZXZ', 'IIIIIIIIIIYZIY', 'IIIIIIIIIIXZIX', 'IIIIIIIIYZZZIY', 'I

In [ ]:
print_circuit = False

ansatz = make_hea_ansatz(hpq.shape[0], print_circuit=print_circuit)
do_vqe(ansatz, hamiltonian, nuclear)

#    1: -27.411471: 0.1270 0.1346 0.5453 0.5618 0.3190 0.2303 0.7547 0.9520 0.5664 0.0317 0.7001 0.4884 0.5417 0.6608 0.1143 0.8263 0.8650 0.6166 0.8257 0.1504 0.7697 0.1895 0.5088 0.1085 0.1938 0.0505 0.7521 0.0037
#    2: -27.411471: 0.1270 0.1346 0.5453 0.5618 0.3190 0.2303 0.7547 0.9520 0.5664 0.0317 0.7001 0.4884 0.5417 0.6608 0.1143 0.8263 0.8650 0.6166 0.8257 0.1504 0.7697 0.1895 0.5088 0.1085 0.1938 0.0505 0.7521 0.0037
#    3: -27.411471: 0.1270 0.1346 0.5453 0.5618 0.3190 0.2303 0.7547 0.9520 0.5664 0.0317 0.7001 0.4884 0.5417 0.6608 0.1143 0.8263 0.8650 0.6166 0.8257 0.1504 0.7697 0.1895 0.5088 0.1085 0.1938 0.0505 0.7521 0.0037
#    4: -27.411471: 0.1270 0.1346 0.5453 0.5618 0.3190 0.2303 0.7547 0.9520 0.5664 0.0317 0.7001 0.4884 0.5417 0.6608 0.1143 0.8263 0.8650 0.6166 0.8257 0.1504 0.7697 0.1895 0.5088 0.1085 0.1938 0.0505 0.7521 0.0037
#    5: -27.411471: 0.1270 0.1346 0.5453 0.5618 0.3190 0.2303 0.7547 0.9520 0.5664 0.0317 0.7001 0.4884 0.5417 0.6608 0.1143 0.8263 0.86

### H2O hea after tapering without picking active space

In [ ]:
hpq, hpqrs, nuclear, nelec, hf = create_integrals_h2o_sto3g()
hamiltonian = create_hamiltonian_quantum_circuit(hpq, hpqrs, JordanWignerMapper())
print(len(hamiltonian))
print(hamiltonian)

converged SCF energy = -74.8801743517023
nOrb : 7
nElec: 10
1086
SparsePauliOp(['IIIIIIIIIIIIII', 'IIIIIIIIIIIIIZ', 'IIIIIIIIIIIIYY', 'IIIIIIIIIIIIXX', 'IIIIIIIIIIYZZY', 'IIIIIIIIIIXZZX', 'IIIIIIIIYZZZZY', 'IIIIIIIIXZZZZX', 'IIIIIIIIIIIIZI', 'IIIIIIIIIIYZYI', 'IIIIIIIIIIXZXI', 'IIIIIIIIYZZZYI', 'IIIIIIIIXZZZXI', 'IIIIIIIIIIIZII', 'IIIIIIIYZZZYII', 'IIIIIIIXZZZXII', 'IIIIIIIIIIZIII', 'IIIIIIIIYZYIII', 'IIIIIIIIXZXIII', 'IIIIIIIIIZIIII', 'IIIIIIIIZIIIII', 'IIIIIIIZIIIIII', 'IIIIIIZIIIIIII', 'IIIIIYYIIIIIII', 'IIIIIXXIIIIIII', 'IIIYZZYIIIIIII', 'IIIXZZXIIIIIII', 'IYZZZZYIIIIIII', 'IXZZZZXIIIIIII', 'IIIIIZIIIIIIII', 'IIIYZYIIIIIIII', 'IIIXZXIIIIIIII', 'IYZZZYIIIIIIII', 'IXZZZXIIIIIIII', 'IIIIZIIIIIIIII', 'YZZZYIIIIIIIII', 'XZZZXIIIIIIIII', 'IIIZIIIIIIIIII', 'IYZYIIIIIIIIII', 'IXZXIIIIIIIIII', 'IIZIIIIIIIIIII', 'IZIIIIIIIIIIII', 'ZIIIIIIIIIIIII', 'IIIIIIIIIIIIZZ', 'IIIIIIIIIIYZYZ', 'IIIIIIIIIIXZXZ', 'IIIIIIIIYZZZYZ', 'IIIIIIIIXZZZXZ', 'IIIIIIIIIIYZIY', 'IIIIIIIIIIXZIX', 'IIIIIIIIYZZZIY', 'I

In [ ]:
z2_symmetries = Z2Symmetries.find_z2_symmetries(hamiltonian)
tapered_op = z2_symmetries.taper(hamiltonian)
print(len(tapered_op))
print(len(tapered_op[1]))
print(tapered_op[1])

16
1035
SparsePauliOp(['IIIIIIIIII', 'ZIZIZIIZZZ', 'ZIZIZIIZZX', 'IIIIIIIIIX', 'IIZIIIZZXI', 'ZIIIZIZIXZ', 'IIIIIIZXII', 'ZIZIZIZXZZ', 'IIIIIIIIIZ', 'ZIIIZIZIYY', 'ZIIIZIZIXX', 'ZIZIZIZYZY', 'ZIZIZIZXZX', 'ZIIIZIZIII', 'ZIZIZIXZZI', 'IIZIIIXZZI', 'IIIIIIIIZI', 'IIZIIIIYYI', 'IIZIIIIXXI', 'IIZIIIIIII', 'IIIIIIIZII', 'IIIIIIZIII', 'ZZZZZZIIII', 'ZZZZZXIIII', 'IIIIIXIIII', 'ZZZXIIIIII', 'IIIXZZIIII', 'ZXIIIIIIII', 'IXZZZZIIII', 'IIIIIZIIII', 'IIIYZYIIII', 'IIIXZXIIII', 'IYZZZYIIII', 'IXZZZXIIII', 'IIIIZIIIII', 'YZZZYIIIII', 'XZZZXIIIII', 'IIIZIIIIII', 'IYZYIIIIII', 'IXZXIIIIII', 'IZIIIIIIII', 'ZIIIIIIIII', 'ZIZIZIIZZI', 'IIZIIIZZXX', 'IIZIIIZZYY', 'IIIIIIZXIX', 'IIIIIIZYIY', 'IIZIIIZZXZ', 'ZIIIZIZIXI', 'IIIIIIZXIZ', 'ZIZIZIZXZI', 'ZIIIZIYIIY', 'IIIIIIXIIX', 'ZIIIZIXIIX', 'ZIZIZIXZZX', 'IIZIIIXZZX', 'ZIZIZIYZZY', 'ZIIIZIIXYY', 'ZIIIZIIXXX', 'ZIIIZIIYYX', 'IIZIIIIYYX', 'IIZIIIIXXX', 'IIZIIIIXYY', 'IIZIIIZZZZ', 'IIIIIIXIIZ', 'ZIIIZIXIIZ', 'IIZIIIZZZX', 'ZIIIZIZIIX', 'ZIZIZIIZXI', 'IIIIIIIIXZ

In [ ]:
print_circuit = False

# ansatz = make_hea_ansatz(hpq.shape[0], print_circuit=print_circuit)
# do_vqe(ansatz, hamiltonian, nuclear)
ansatz = make_odd_hea_ansatz(n_spin=nelec, reps=2, print_circuit=False)
do_vqe(ansatz, tapered_op[6], nuclear)

#    1: -44.392589: 0.3778 0.1996 0.1276 0.5268 0.9595 0.8772 0.9167 0.1772 0.8787 0.8696 0.8890 0.5711 0.6563 0.6140 0.0157 0.4965 0.1323 0.1824 0.6206 0.4298
#    2: -44.392589: 0.3778 0.1996 0.1276 0.5268 0.9595 0.8772 0.9167 0.1772 0.8787 0.8696 0.8890 0.5711 0.6563 0.6140 0.0157 0.4965 0.1323 0.1824 0.6206 0.4298
#    3: -44.392589: 0.3778 0.1996 0.1276 0.5268 0.9595 0.8772 0.9167 0.1772 0.8787 0.8696 0.8890 0.5711 0.6563 0.6140 0.0157 0.4965 0.1323 0.1824 0.6206 0.4298
#    4: -44.392589: 0.3778 0.1996 0.1276 0.5268 0.9595 0.8772 0.9167 0.1772 0.8787 0.8696 0.8890 0.5711 0.6563 0.6140 0.0157 0.4965 0.1323 0.1824 0.6206 0.4298
#    5: -44.392589: 0.3778 0.1996 0.1276 0.5268 0.9595 0.8772 0.9167 0.1772 0.8787 0.8696 0.8890 0.5711 0.6563 0.6140 0.0157 0.4965 0.1323 0.1824 0.6206 0.4298
#    6: -44.392589: 0.3778 0.1996 0.1276 0.5268 0.9595 0.8772 0.9167 0.1772 0.8787 0.8696 0.8890 0.5711 0.6563 0.6140 0.0157 0.4965 0.1323 0.1824 0.6206 0.4298
#    7: -44.392589: 0.3778 0.1996 0.1276

### H2O hea after tapering with picking active space

In [ ]:
hpq, hpqrs, nuclear, nelec, hf = create_integrals_h2o_sto3g()
active_core = 4
active_number = 2
nact, act_nelec, nuclearAddEcore, hpq, hpqrs, nuclear = active_space_unpolarized(hf, ncore=active_core, nact=active_number)
hamiltonian = create_hamiltonian_quantum_circuit(hpq, hpqrs, JordanWignerMapper())
print(len(hamiltonian))
print(hamiltonian)

converged SCF energy = -74.8801743517024
nOrb : 7
nElec: 10
15
SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.87206336+0.j,  0.11923749+0.j, -0.17865262+0.j,  0.11923749+0.j,
 -0.17865262+0.j,  0.14753433+0.j,  0.22003977+0.j,  0.00963433+0.j,
  0.00963433+0.j,  0.00963433+0.j,  0.00963433+0.j,  0.15716866+0.j,
  0.15716866+0.j,  0.16144783+0.j,  0.14753433+0.j])


In [ ]:
z2_symmetries = Z2Symmetries.find_z2_symmetries(hamiltonian)
tapered_op = z2_symmetries.taper(hamiltonian)
print(len(tapered_op))
print(len(tapered_op[1]))
print(tapered_op[1])

8
2
SparsePauliOp(['I', 'Z'],
              coeffs=[-0.9306553 +0.j, -0.35730523+0.j])


In [ ]:
print_circuit = False

# ansatz = make_hea_ansatz(hpq.shape[0], print_circuit=print_circuit)
# do_vqe(ansatz, hamiltonian, nuclear)
ansatz = make_odd_hea_ansatz(n_spin=act_nelec-1, reps=2, print_circuit=False)
do_vqe(ansatz, tapered_op[6], nuclear)

# the decomposed circuit is over-simplified to run properly 

#    1: 8.546657: 0.0330 0.8472
#    2: 8.546657: 0.0330 0.8472
#    3: 8.546657: 0.0330 0.8472
Energy: 8.546656884469115


### C2H4 hea before tapering 

In [ ]:
hpq, hpqrs, nuclear, nelec, hf = create_integrals_c2h4_sto3g()
hamiltonian = create_hamiltonian_quantum_circuit(hpq, hpqrs, JordanWignerMapper())
# HEAtest = make_hea_ansatz(n_spatial_orb=arr_active[0], reps=2, print_circuit=False)
# VQEtest = do_vqe(HEAtest, hamiltonian, arr_active[2])
print(len(hamiltonian))
print(hamiltonian)

converged SCF energy = -77.0720875637473
nOrb : 14
nElec: 16
8919
SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIYZY', 'IIIIIIIIIIIIIIIIIIIIIIIIIXZX', 'IIIIIIIIIIIIIIIIIIIIIIYZZZZY', 'IIIIIIIIIIIIIIIIIIIIIIXZZZZX', 'IIIIIIIIIIIIIIIIIYZZZZZZZZZY', 'IIIIIIIIIIIIIIIIIXZZZZZZZZZX', 'IIIIIIIIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIIIIIIYZYI', 'IIIIIIIIIIIIIIIIIIIIIIIIXZXI', 'IIIIIIIIIIIIIIIIYZZZZZZZZZYI', 'IIIIIIIIIIIIIIIIXZZZZZZZZZXI', 'IIIIIIIIIIIIIIYZZZZZZZZZZZYI', 'IIIIIIIIIIIIIIXZZZZZZZZZZZXI', 'IIIIIIIIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIIIIIYZZYII', 'IIIIIIIIIIIIIIIIIIIIIIXZZXII', 'IIIIIIIIIIIIIIIIIYZZZZZZZYII', 'IIIIIIIIIIIIIIIIIXZZZZZZZXII', 'IIIIIIIIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIYZZZZZZZYIII', 'IIIIIIIIIIIIIIIIXZZZZZZZXIII', 'IIIIIIIIIIIIIIYZZZZZZZZZYIII', 'IIIIIIIIIIIIIIXZZZZZZZZZXIII', 'IIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIYZZZZYIIII', 'IIIIIIIIIIIIIIIIIIXZZZZXIIII', 'IIIIIIIIIIIIIIIIIIIIII

In [ ]:
print_circuit = False

ansatz = make_hea_ansatz(hpq.shape[0], print_circuit=print_circuit)
do_vqe(ansatz, hamiltonian, nuclear)

## to long to run 

### C2H4 hea after tapering without picking active space 

In [ ]:
hpq, hpqrs, nuclear, nelec, hf = create_integrals_c2h4_sto3g()
hamiltonian = create_hamiltonian_quantum_circuit(hpq, hpqrs, JordanWignerMapper())
print(len(hamiltonian))
print(hamiltonian)

converged SCF energy = -77.0720875637474
nOrb : 14
nElec: 16
8919
SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIYZY', 'IIIIIIIIIIIIIIIIIIIIIIIIIXZX', 'IIIIIIIIIIIIIIIIIIIIIIYZZZZY', 'IIIIIIIIIIIIIIIIIIIIIIXZZZZX', 'IIIIIIIIIIIIIIIIIYZZZZZZZZZY', 'IIIIIIIIIIIIIIIIIXZZZZZZZZZX', 'IIIIIIIIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIIIIIIYZYI', 'IIIIIIIIIIIIIIIIIIIIIIIIXZXI', 'IIIIIIIIIIIIIIIIYZZZZZZZZZYI', 'IIIIIIIIIIIIIIIIXZZZZZZZZZXI', 'IIIIIIIIIIIIIIYZZZZZZZZZZZYI', 'IIIIIIIIIIIIIIXZZZZZZZZZZZXI', 'IIIIIIIIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIIIIIYZZYII', 'IIIIIIIIIIIIIIIIIIIIIIXZZXII', 'IIIIIIIIIIIIIIIIIYZZZZZZZYII', 'IIIIIIIIIIIIIIIIIXZZZZZZZXII', 'IIIIIIIIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIYZZZZZZZYIII', 'IIIIIIIIIIIIIIIIXZZZZZZZXIII', 'IIIIIIIIIIIIIIYZZZZZZZZZYIII', 'IIIIIIIIIIIIIIXZZZZZZZZZXIII', 'IIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIYZZZZYIIII', 'IIIIIIIIIIIIIIIIIIXZZZZXIIII', 'IIIIIIIIIIIIIIIIIIIIII

In [ ]:
z2_symmetries = Z2Symmetries.find_z2_symmetries(hamiltonian)
tapered_op = z2_symmetries.taper(hamiltonian)
print(len(tapered_op))
print(len(tapered_op[1]))
print(tapered_op[1])

32
8914
SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIII', 'ZIZIZIZIIZZIZIZIZIZZZIZ', 'IZIIZZZZIZIIIZIZZIIIZZX', 'ZZZIIZIZIIZIZZZZIIZZIZX', 'IIIIIZZIIIIIIZZZZZIZXII', 'ZIZIZZIIIZZIZZIZIZZIXIZ', 'IIIIIIIIIIIIIZZZXIIIIII', 'ZIZIZIZIIZZIZZIZXIZZZIZ', 'ZZZIIZIZIIZIZZZZIIZZIZI', 'ZZZIIZIZIIZIZZZZIIZZIXZ', 'IIIIIIIIIIIIIIIIIIIIIXZ', 'ZIZIZIZIIZZIZZIXZIZZZIZ', 'IZIIZZZZIZIIIIZXZIIIZZZ', 'ZIZIZIZIIZZIZXZIZIZZZIZ', 'IZIIZZZZIZIIIXIZZIIIZZZ', 'IIIIIIIIIIIIIIIIIIIIIIZ', 'IZIIZIIZIZIIIIZIIZIZYZY', 'IZIIZIIZIZIIIIZIIZIZXZX', 'IZIIZZZZIZIIIIZIYIIIZZY', 'IZIIZZZZIZIIIIZIXIIIZZX', 'IIIIIIIIIIIIIIIIIIIIIZI', 'IZIIZZZZIZIIIIZYZIIIZYI', 'IZIIZZZZIZIIIIZXZIIIZXI', 'IZIIZZZZIZIIIYIZZIIIZYI', 'IZIIZZZZIZIIIXIZZIIIZXI', 'IZIIZIIZIZIIIIZIIZIZIII', 'IZIIZZZZIZIIIIZIIXIIZII', 'IIIIIZZIIIIIIIIIIXIZZII', 'IIIIIIIIIIIIIIIIIIIIZII', 'IIIIIZZIIIIIIIIIYZIZYII', 'IIIIIZZIIIIIIIIIXZIZXII', 'IIIIIIIIIIIIIIIIIIIZIII', 'IIIIIZZIIIIIIIYZZZIYIII', 'IIIIIZZIIIIIIIXZZZIXIII', 'IIIIIZZIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIZIIII', 'IIII

In [ ]:
print_circuit = False

# ansatz = make_hea_ansatz(hpq.shape[0], print_circuit=print_circuit)
# do_vqe(ansatz, hamiltonian, nuclear)
ansatz = make_odd_hea_ansatz(n_spin=nelec, reps=2, print_circuit=False)
do_vqe(ansatz, tapered_op[6], nuclear)

#    1: -39.627259: 0.9294 0.4550 0.5521 0.2539 0.2072 0.9871 0.6503 0.2208 0.9465 0.0365 0.0250 0.4501 0.2626 0.1801 0.8588 0.4330 0.1869 0.2128 0.8664 0.6721
#    2: -39.627259: 0.9294 0.4550 0.5521 0.2539 0.2072 0.9871 0.6503 0.2208 0.9465 0.0365 0.0250 0.4501 0.2626 0.1801 0.8588 0.4330 0.1869 0.2128 0.8664 0.6721
#    3: -39.627259: 0.9294 0.4550 0.5521 0.2539 0.2072 0.9871 0.6503 0.2208 0.9465 0.0365 0.0250 0.4501 0.2626 0.1801 0.8588 0.4330 0.1869 0.2128 0.8664 0.6721
#    4: -39.627259: 0.9294 0.4550 0.5521 0.2539 0.2072 0.9871 0.6503 0.2208 0.9465 0.0365 0.0250 0.4501 0.2626 0.1801 0.8588 0.4330 0.1869 0.2128 0.8664 0.6721
#    5: -39.627259: 0.9294 0.4550 0.5521 0.2539 0.2072 0.9871 0.6503 0.2208 0.9465 0.0365 0.0250 0.4501 0.2626 0.1801 0.8588 0.4330 0.1869 0.2128 0.8664 0.6721
#    6: -39.627259: 0.9294 0.4550 0.5521 0.2539 0.2072 0.9871 0.6503 0.2208 0.9465 0.0365 0.0250 0.4501 0.2626 0.1801 0.8588 0.4330 0.1869 0.2128 0.8664 0.6721
#    7: -39.627259: 0.9294 0.4550 0.5521

### C2H4 hea after tapering with picking active space 

In [ ]:
hpq, hpqrs, nuclear, nelec, hf = create_integrals_c2h4_sto3g()
active_core = 7
active_number = 2
nact, act_nelec, nuclearAddEcore, hpq, hpqrs, nuclear = active_space_unpolarized(hf, ncore=active_core, nact=active_number)
print ("nOrb : ", nact)
print ("nElec: ", act_nelec)
hamiltonian = create_hamiltonian_quantum_circuit(hpq, hpqrs, JordanWignerMapper())
print(len(hamiltonian))
print(hamiltonian)

converged SCF energy = -77.0720875637473
nOrb : 14
nElec: 16
nOrb :  2
nElec:  2
15
SparsePauliOp(['IIII', 'IIIZ', 'IIZI', 'IZII', 'ZIII', 'IIZZ', 'IZIZ', 'YYYY', 'XXYY', 'YYXX', 'XXXX', 'ZIIZ', 'IZZI', 'ZIZI', 'ZZII'],
              coeffs=[-0.6782873 +0.j,  0.07696473+0.j, -0.07816186+0.j,  0.07696473+0.j,
 -0.07816186+0.j,  0.08422202+0.j,  0.12687993+0.j,  0.0430091 +0.j,
  0.0430091 +0.j,  0.0430091 +0.j,  0.0430091 +0.j,  0.12723111+0.j,
  0.12723111+0.j,  0.13089538+0.j,  0.08422202+0.j])


In [ ]:
z2_symmetries = Z2Symmetries.find_z2_symmetries(hamiltonian)
tapered_op = z2_symmetries.taper(hamiltonian)
print(len(tapered_op))
print(len(tapered_op[1]))
print(tapered_op[1])

8
2
SparsePauliOp(['I', 'Z'],
              coeffs=[-0.67427185+0.j, -0.15632371+0.j])


In [ ]:
print_circuit = False

ansatz = make_odd_hea_ansatz(n_spin=act_nelec-1, reps=2, print_circuit=False)
do_vqe(ansatz, tapered_op[6], nuclear)

# the decomposed circuit is over-simplified to run properly 

#    1: 32.243875: 0.4572 0.2846
#    2: 32.243875: 0.4572 0.2846
#    3: 32.243875: 0.4572 0.2846
Energy: 32.24387495410524
